## 0. Introduction
Objectif: Fine-tuner le modèle XGBoost en explorant différents hyperparamètres.

Traçabilité et reproductibilité via MLflow.

## 1. Import & configuration

In [11]:
import sys, os
import mlflow
import mlflow.xgboost
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    RocCurveDisplay,
    PrecisionRecallDisplay,
)
from imblearn.over_sampling import SMOTE
import seaborn as sns
from matplotlib import pyplot as plt
from xgboost import XGBClassifier
import pandas as pd
from rich import print
import shap

# Dataset + constantes
sys.path.insert(0, "..")
from config import PROCESSED_DATA_PATH, TARGET_COL, SEED, FIG_DIR, CV_STRATEGY
from utils import compute_train_test_metrics

df = pd.read_csv(PROCESSED_DATA_PATH)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# --- Identifier les colonnes numériques et binaires
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
binary_cols = [col for col in numeric_cols if X[col].nunique() == 2 or (X[col].dtype == 'int64' and set(X[col].unique()) == {0, 1})]
numeric_cols = [col for col in numeric_cols if col not in binary_cols]

NUMERIC_COLS = numeric_cols
BINARY_COLS = binary_cols

print(f"[bold cyan]📊 Colonnes détectées par feature engineering:[/bold cyan]")
print(f"   - Numériques: {NUMERIC_COLS}")
print(f"   - Binaires: {BINARY_COLS}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# Configuration MLflow
mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("Rugby Kicks - XGBoost Finetuning")
mlflow.set_experiment_tag("author", "Xavier")

# Fin de tout run actif avant de commencer
if mlflow.active_run():
    mlflow.end_run()

# Préparer les datasets
X_trainfull = pd.concat([X_train, y_train], axis=1)
X_testfull = pd.concat([X_test, y_test], axis=1)

train_dataset = mlflow.data.from_pandas(X_trainfull, source=PROCESSED_DATA_PATH, name="Training_set")
test_dataset = mlflow.data.from_pandas(X_testfull, source=PROCESSED_DATA_PATH, name="Test_set")

print("[bold green]✅ Configuration prête[/bold green]")

📊 Colonnes détectées par feature engineering:

- Numériques: ['time_norm', 'distance', 'angle', 'wind_speed', 'precipitation_probability']

- Binaires: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 'has_previous_attempts']

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/data/processed/kicks_ready_for_model.csv'. Exception: 
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/xaviercou

✅ Configuration prête

## 2. Preprocessing & Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Créer la pipeline de preprocessing
preprocessing_pipeline = ColumnTransformer([
    ('scaler', StandardScaler(), NUMERIC_COLS),
    ('passthrough', 'passthrough', BINARY_COLS)
], remainder='drop')

# FIT sur X_train UNIQUEMENT (prevent data leakage!)
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_test_processed = preprocessing_pipeline.transform(X_test)

# Convertir en DataFrame pour maintenabilité
feature_names = NUMERIC_COLS + BINARY_COLS
X_train_processed = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print("[bold green]✅ Pipeline de preprocessing créée et appliquée[/bold green]")
print(f"   - Colonnes numériques standardisées: {NUMERIC_COLS}")
print(f"   - Colonnes binaires conservées: {BINARY_COLS}")
print(f"   - X_train shape: {X_train_processed.shape}")
print(f"   - X_test shape: {X_test_processed.shape}")

✅ Pipeline de preprocessing créée et appliquée

- Colonnes numériques standardisées: ['time_norm', 'distance', 'angle', 'wind_speed', 
'precipitation_probability']

- Colonnes binaires conservées: ['is_left_footed', 'game_away', 'is_endgame', 'is_start', 'is_left_side', 
'has_previous_attempts']

- X_train shape: (13873, 11)

- X_test shape: (3469, 11)

## 2.5 SMOTE - Synthetic Minority Over-sampling

In [13]:
# --- Appliquer SMOTE pour équilibrer les classes
print("[bold cyan]🔄 Application de SMOTE (Synthetic Minority Over-sampling)...[/bold cyan]")

# Afficher la distribution AVANT SMOTE
print(f"\n[bold]Avant SMOTE:[/bold]")
print(f"  - Classe 0: {(y_train == 0).sum()} samples")
print(f"  - Classe 1: {(y_train == 1).sum()} samples")
print(f"  - Ratio: {(y_train == 0).sum() / (y_train == 1).sum():.2f}:1")

# Appliquer SMOTE
smote = SMOTE(random_state=SEED)
X_train_smote, y_train_smote = smote.fit_resample(X_train_processed, y_train)

# Afficher la distribution APRÈS SMOTE
print(f"\n[bold]Après SMOTE:[/bold]")
print(f"  - Classe 0: {(y_train_smote == 0).sum()} samples")
print(f"  - Classe 1: {(y_train_smote == 1).sum()} samples")
print(f"  - Ratio: {(y_train_smote == 0).sum() / (y_train_smote == 1).sum():.2f}:1")

print(f"\n[bold green]✅ SMOTE appliqué avec succès[/bold green]")
print(f"  X_train_smote shape: {X_train_smote.shape}")
print(f"  y_train_smote shape: {y_train_smote.shape}")

🔄 Application de SMOTE (Synthetic Minority Over-sampling)...

Avant SMOTE:

- Classe 0: 3271 samples

- Classe 1: 10602 samples

- Ratio: 0.31:1

...

- Classe 1: 10602 samples

- Ratio: 1.00:1

✅ SMOTE appliqué avec succès

X_train_smote shape: (21204, 11)

y_train_smote shape: (21204,)

## 3. XGBoost Fine-tuning with GridSearchCV

In [14]:
# Calcul du poids de la classe majoritaire / minoritaire
# neg, pos = np.bincount(y_train)
scale_pos_weight = 0.7

# --- Hyperparamètres de base (à partir du modèle benchmark)
base_params = {
    "scale_pos_weight": scale_pos_weight,
    "random_state": SEED,
    "eval_metric": "logloss",
    "n_jobs": -1,
}

# --- Grille de paramètres RÉDUITE (Option 1: 16 combos au lieu de 729)
# Focus sur les hyperparamètres clés, moins de valeurs testées
param_grid = {
    "n_estimators": [250, 350],           # 2 valeurs (au lieu de 3)
    "max_depth": [5, 6],                  # 2 valeurs (au lieu de 3)
    "learning_rate": [0.05, 0.1],         # 2 valeurs (au lieu de 3)
    "subsample": [0.8, 0.9],              # 2 valeurs (au lieu de 3)
    "colsample_bytree": [0.8],            # 1 valeur (au lieu de 3) - bon équilibre
    "min_child_weight": [3],              # 1 valeur (au lieu de 3) - valeur benchmark
}

# Calcul du nombre de combinaisons
total_combos = np.prod([len(v) for v in param_grid.values()])
total_cv_fits = total_combos * CV_STRATEGY.get_n_splits()

print("[bold cyan]🔍 Grille de paramètres RÉDUITE pour RandomizedSearchCV:[/bold cyan]")
for param, values in param_grid.items():
    print(f"   - {param}: {values}")
print(f"\n   ✅ Total combinations: {total_combos}")
print(f"   ✅ Total CV fits (avec CV={CV_STRATEGY.get_n_splits()}): {total_cv_fits}")
print(f"   ✅ ~85% moins de combinaisons qu'avant!")

🔍 Grille de paramètres RÉDUITE pour RandomizedSearchCV:

- n_estimators: [250, 350]

- max_depth: [5, 6]

- learning_rate: [0.05, 0.1]

- subsample: [0.8, 0.9]

- colsample_bytree: [0.8]

- min_child_weight: [3]

✅ Total combinations: 16

✅ Total CV fits (avec CV=5): 80

✅ ~85% moins de combinaisons qu'avant!

In [ ]:
with mlflow.start_run(
    run_name="XGBoost_GridSearchCV_SMOTE_Optimized_scale_pos_weight_0.7",
    description="Fine-tuning XGBoost avec grille réduite + SMOTE (16 combos) avec scale_pos_weight=0.7",
):

    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(test_dataset, context="testing")
    mlflow.set_tag("model_type", "xgboost_finetuned")
    mlflow.set_tag("optimization_method", "gridsearchcv")
    mlflow.set_tag("resampling", "SMOTE")
    mlflow.set_tag("search_strategy", "reduced_grid_option1")

    # --- Logger les paramètres de base
    base_params_log = base_params.copy()
    base_params_log["scale_pos_weight"] = round(scale_pos_weight, 2)
    mlflow.log_params({f"base_{k}": str(v) for k, v in base_params_log.items()})
    
    # --- Logger la grille de paramètres
    for param, values in param_grid.items():
        mlflow.log_param(f"grid_{param}", str(values))

    # --- GridSearchCV
    xgb_base = XGBClassifier(**base_params)
    grid_search = GridSearchCV(
        estimator=xgb_base,
        param_grid=param_grid,
        cv=CV_STRATEGY,
        scoring="f1_weighted",  # Utiliser F1 pondéré pour imbalanced data
        n_jobs=-1,
        verbose=1
    )

    print("[bold cyan]🚀 Lancement de GridSearchCV sur données SMOTE (grille réduite)...[/bold cyan]")
    grid_search.fit(X_train_smote, y_train_smote)

    # --- Récupérer le meilleur modèle
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    best_cv_score = grid_search.best_score_

    print(f"[bold green]✅ GridSearchCV terminé[/bold green]")
    print(f"   - Best CV Score (F1_weighted sur SMOTE): {best_cv_score:.4f}")
    print(f"[bold yellow]🏆 Best Parameters found:[/bold yellow]")
    for param, value in best_params.items():
        print(f"   - {param}: {value}")
    
    # --- Logger les meilleurs paramètres
    mlflow.log_params({f"best_{k}": str(v) for k, v in best_params.items()})
    mlflow.log_metric("best_cv_score_f1_weighted_smote", best_cv_score)

    # --- Prédictions finales
    # ⚠️  IMPORTANT: Les prédictions utilisent les données originales (non résamplées)
    y_pred_train = best_model.predict(X_train_processed)
    y_proba_train = best_model.predict_proba(X_train_processed)[:, 1]
    
    y_pred = best_model.predict(X_test_processed)
    y_proba = best_model.predict_proba(X_test_processed)[:, 1]

    # --- Métriques train/test + overfitting gaps
    final_metrics_test, cm, (fp_rate, fn_rate) = compute_train_test_metrics(
        y_train, y_pred_train, y_proba_train,
        y_test, y_pred, y_proba
    )
    
    # --- Log metrics
    mlflow.log_metrics(final_metrics_test)

    # --- Log du modèle
    mlflow.xgboost.log_model(
        xgb_model=best_model,
        name="xgboost_finetuned_smote_optimized_model",
        input_example=X_test_processed.iloc[:5],
    )

    # --- Matrice de confusion
    plt.figure()
    sns.heatmap(cm, annot=True, fmt="d", cmap="mako", cbar=False)
    plt.title("Confusion Matrix - XGBoost Finetuned (SMOTE + Optimized)")
    plt.xlabel("Prédictions")
    plt.ylabel("Vérités terrain")
    plt.tight_layout()
    cm_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_confusion_matrix.png"
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)
    plt.close()

    # --- Courbe ROC
    RocCurveDisplay.from_estimator(best_model, X_test_processed, y_test)
    plt.title("ROC Curve - XGBoost Finetuned (SMOTE + Optimized)")
    plt.tight_layout()
    roc_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_roc_curve.png"
    plt.savefig(roc_path)
    mlflow.log_artifact(roc_path)
    plt.close()

    # --- Courbe Precision-Recall
    PrecisionRecallDisplay.from_estimator(best_model, X_test_processed, y_test)
    plt.title("Precision-Recall Curve - XGBoost Finetuned (SMOTE + Optimized)")
    plt.tight_layout()
    pr_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_pr_curve.png"
    plt.savefig(pr_path)
    mlflow.log_artifact(pr_path)
    plt.close()

    # --- Importance des features
    importances = pd.Series(best_model.feature_importances_, index=X.columns).sort_values(
        ascending=False
    )
    plt.figure()
    imp_df = pd.DataFrame({'importance': importances[:10].values, 'feature': importances[:10].index})
    sns.barplot(data=imp_df, x='importance', y='feature', palette="rocket", hue='feature', legend=False)
    plt.title("Top 10 Feature Importances - XGBoost Finetuned (SMOTE + Optimized)")
    plt.tight_layout()
    feat_imp_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_feature_importances.png"
    plt.savefig(feat_imp_path)
    mlflow.log_artifact(feat_imp_path)
    plt.close()

    print("[bold cyan]✅ XGBoost Finetuned (SMOTE + Optimized) logué dans MLflow avec succès[/bold cyan]")

    # ========== SHAP Analysis - DANS LE MÊME RUN ==========
    print("\n[bold magenta]🔍 SHAP Analysis - Model Interpretability[/bold magenta]")

    # Simple feature importance approximation using model's native feature importance
    print("\n[cyan]Computing feature importance (Fast method)...[/cyan]")

    importances_shap = pd.Series(
        best_model.feature_importances_, 
        index=X_test_processed.columns
    ).sort_values(ascending=False)

    # 1️⃣ SUMMARY BAR PLOT - Global Feature Importance
    print("\n[cyan]1. Summary Bar Plot - Top 10 Features[/cyan]")
    plt.figure()
    top_n = 10
    importances_shap.head(top_n).plot(kind='barh', color='steelblue')
    plt.xlabel('Feature Importance')
    plt.ylabel('Features')
    plt.title("Feature Importance - XGBoost Finetuned (SMOTE + Optimized)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    shap_bar_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_shap_bar_plot.png"
    plt.savefig(shap_bar_path)
    plt.close()
    print(f"   ✅ Saved: {shap_bar_path}")
    mlflow.log_artifact(shap_bar_path)

    # 2️⃣ FEATURE IMPORTANCE DISTRIBUTION
    print("\n[cyan]2. Feature Importance Distribution[/cyan]")
    plt.figure()
    importances_shap.plot(kind='bar', color='coral', alpha=0.7)
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title("All Features Importance Distribution")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    shap_dot_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_shap_dot_plot.png"
    plt.savefig(shap_dot_path)
    plt.close()
    print(f"   ✅ Saved: {shap_dot_path}")
    mlflow.log_artifact(shap_dot_path)

    # 3️⃣ PREDICTION EXPLANATION (First sample)
    print("\n[cyan]3. Prediction Explanation (First Test Sample)[/cyan]")
    first_pred = y_pred[0]
    first_proba = y_proba[0]
    actual_label = y_test.iloc[0]

    fig, ax = plt.subplots()
    top_features = importances_shap.head(10).index
    top_importance = importances_shap.head(10).values
    colors = ['green' if x > 0 else 'red' for x in top_importance]

    ax.barh(range(len(top_features)), top_importance, color=colors, alpha=0.7)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features)
    ax.set_xlabel('Feature Importance', fontsize=12)
    ax.set_title(f"Prediction {0}: {first_pred} (Confidence: {first_proba:.2%}) | Actual: {actual_label}", 
                 fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    plt.tight_layout()
    shap_force_path = f"{FIG_DIR}/xgb_finetuned_smote_optimized_shap_force_first_prediction.png"
    plt.savefig(shap_force_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Saved: {shap_force_path}")
    mlflow.log_artifact(shap_force_path)

    # 4️⃣ FEATURE IMPORTANCE SUMMARY
    print("\n[cyan]4. Feature Importance Summary[/cyan]")
    print("\nTop 5 Most Important Features:")
    for idx, (feature, importance) in enumerate(importances_shap.head(5).items(), 1):
        print(f"   {idx}. {feature:30s}: {importance:.6f}")

    print("\n[green]✅ SHAP artifacts logged to MLflow in same run![/green]")
    print("\n[bold green]✨ SHAP Analysis Complete! (⚡ Fast version)[/bold green]")

# 🔎 Résumé local
print(
    f"""
📊 Résumé des métriques - XGBoost Finetuned (SMOTE + Optimized) :
  Train Accuracy: {final_metrics_test['train_accuracy']:.3f}
  Test Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  Overfitting Gap (Accuracy): {final_metrics_test['overfit_gap_accuracy']:.3f}
  ---
  Train F1 classe 0: {final_metrics_test['train_f1_0']:.3f}
  Test F1 classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Overfitting Gap (F1_0): {final_metrics_test['overfit_gap_f1_0']:.3f}
  ---
  Accuracy:  {final_metrics_test['final_test_accuracy']:.3f}
  ROC-AUC:   {final_metrics_test['final_test_auc']:.3f}
  Precision classe 0: {final_metrics_test['final_test_precision_0']:.3f}
  Recall classe 0:    {final_metrics_test['final_test_recall_0']:.3f}
  F1-score classe 0:  {final_metrics_test['final_test_f1_0']:.3f}
  Precision classe 1: {final_metrics_test['final_test_precision_1']:.3f}
  Recall classe 1:    {final_metrics_test['final_test_recall_1']:.3f}
  F1-score classe 1:  {final_metrics_test['final_test_f1_1']:.3f}
  FPR:       {fp_rate:.3f}
  FNR:       {fn_rate:.3f}
"""
)

/Users/xaviercoulon/Documents/OC/OC_P6_Rugby_MLOps/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🚀 Lancement de GridSearchCV sur données SMOTE (grille réduite)...

Fitting 5 folds for each of 16 candidates, totalling 80 fits


✅ GridSearchCV terminé

- Best CV Score (F1_weighted sur SMOTE): 0.8119

...

4. has_previous_attempts         : 0.064203

5. game_away                     : 0.064155

✅ SHAP artifacts logged to MLflow in same run!

✨ SHAP Analysis Complete! (⚡ Fast version)

📊 Résumé des métriques - XGBoost Finetuned (SMOTE + Optimized) :
  Train Accuracy: 0.850
  Test Accuracy:  0.730
  Overfitting Gap (Accuracy): 0.121
  ---
  Train F1 classe 0: 0.712
  Test F1 classe 0:  0.463
  Overfitting Gap (F1_0): 0.249
  ---
  Accuracy:  0.730
  ROC-AUC:   0.767
  Precision classe 0: 0.435
  Recall classe 0:    0.495
  F1-score classe 0:  0.463
  Precision classe 1: 0.837
  Recall classe 1:    0.802
  F1-score classe 1:  0.819
  FPR:       0.505
  FNR:       0.198